# Test FSI Problem with Different Models



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import torch
import numpy as np
import gc

import h5py
import pandas as pd
from matplotlib import pyplot as plt

Import local packages

In [3]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../.."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.utils.logger import Logging
from src.utils.utils import lp_error
from src.utils.logger import Logging

from src.utils.plot_loss import plot_loss_history

Set various constant variables: model path, name, etc.

# Create logger

In [4]:
TEST_DATA_PKL = "../../data/IB_PINN3.mat"
TEST_CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "result/fsi")

logger = Logging(TEST_CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()

In [5]:
RBA = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-18_15-52-02-172851/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-16_18-05-23-054477/model.pth"
FIXED_WEIGHT = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-18_14-27-37-153943/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-17_17-46-43-410794/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-16_17-28-42-573363/model.pth"
SA = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-18_15-25-41-410079/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-16_18-32-58-880576/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-33-081291/model.pth"
Grad_stat = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-18_15-46-19-868599/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-16_17-51-22-642694/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-23-782676/model.pth"

MODEL_PATH_LIST = {
    "RBA": RBA,
    "Fixed": FIXED_WEIGHT,
    "SA": SA,
    "grad_stat": Grad_stat,
}


SOLVER_TO_MODULE = {
    "tanh": "src.nn.tanh",
    "xsig": "src.nn.xsigmoid",
}

DEVICE = "cpu"

## Load the Model and Test

In [6]:
from src.data.IBM_data_loader import (
    generate_fluid_testing_dataset,
    generate_solid_testing_dataset,
)

TEST_DATA_PATH = "../../../data/IB_PINN3.mat"


[time_, xfa, yfa, ufa, vfa, pfa, fxfa, fyfa] = generate_fluid_testing_dataset(
    TEST_DATA_PATH
)

In [7]:
for method, model_path in MODEL_PATH_LIST.items():
    logger.print(f"MODEL_PATH {model_path}")
    # Load the state from the saved model
    state = torch.load(
        model_path,
    )
    config = state.get("config", {})
    solver = config.get("solver")

    # Extract model configuration from state
    model_activation_name = config.get("activation", "Tanh")
    model_architecture = config.get("network")
    loss_history = state.get("loss_history")
    data_mean = state.get("data_mean")
    data_std = state.get("data_std")
    data_mean = data_mean.to(DEVICE)
    data_std = data_std.to(DEVICE)

    # Dynamically import the correct module and class
    if solver in SOLVER_TO_MODULE:
        module = __import__(SOLVER_TO_MODULE[solver], fromlist=["PINNKAN"])
        PINNKAN = getattr(module, "PINNKAN")

        # Initialize fluid and solid models
        model = PINNKAN(model_architecture, model_activation_name).to("cpu")
    else:
        raise ValueError(f"Solver {solver} not supported")
    model.load_state_dict(state["model_state_dict"])
    logger.print(f"model loaded from {model_path}")
    logger.print(f"problem: {config.get('problem')}")
    logger.print(f"dataset_path:  , {config.get('dataset_path')}")
    logger.print(f"batch size:  , {config.get('batch_size')}")
    logger.print(f"network:  , {config.get('network')}")
    logger.print(f"term loss weights :  , {config.get('weights')}")
    logger.print(f"method:  , {config.get('weighting')}")
    logger.print(f"solver:  , {config.get('solver')}")
    logger.print(
        f"number of iterations:  , {len(loss_history[next(iter(loss_history))])}"
    )

    test_torch_data = torch.tensor(
        np.concatenate([time_, xfa, yfa], axis=1), dtype=torch.float32
    ).to(DEVICE)
    with torch.no_grad():
        predictions = model.forward(test_torch_data, data_mean[:3], data_std[:3])
    if predictions.is_cuda:
        predictions = predictions.cpu()
    u_pred = predictions[:, 0:1].numpy()
    v_pred = predictions[:, 1:2].numpy()
    p_pred = predictions[:, 2:3].numpy()
    fx_pred = predictions[:, 3:4].numpy()
    fy_pred = predictions[:, 4:5].numpy()

    text = "RelL2_"
    # logger.print("\n Relative L2 ERROR:")
    u_error2 = lp_error(u_pred, ufa, (text + "U%"), logger, 2)
    v_error2 = lp_error(v_pred, vfa, (text + "V%"), logger, 2)
    p_error2 = lp_error(p_pred, pfa, (text + "P%"), logger, 2)
    fx_error2 = lp_error(fx_pred, fxfa, (text + "Fx%"), logger, 2)
    fx_error2 = lp_error(fy_pred, fyfa, (text + "Fx%"), logger, 2)

    for key in loss_history:
        logger.print("Final loss %s: %e" % (key, loss_history[key][-1]))

    final_loss_sum = sum([l[-1] for l in loss_history.values()])
    logger.print(f"final loss sum: {final_loss_sum:.2e}")

    logger.print("******************************\n")

    # Delete model and clear cache after each iteration
    del model
    torch.cuda.empty_cache()
    del predictions
    del state

    gc.collect()  # Force garbage collection to release memory

logger.print("file directory:", logger.get_output_dir())

INFO:src.utils.logger:MODEL_PATH /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-17_18-18-48-029989/model.pth


INFO:src.utils.logger:model loaded from /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-17_18-18-48-029989/model.pth
INFO:src.utils.logger:problem: fsi
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 128
INFO:src.utils.logger:network:  , [3, 400, 400, 400, 400, 5]
INFO:src.utils.logger:term loss weights :  , None
INFO:src.utils.logger:method:  , RBA
INFO:src.utils.logger:solver:  , xsig
INFO:src.utils.logger:number of iterations:  , 60001
INFO:src.utils.logger:RelL2_U%  : 47.30 
INFO:src.utils.logger:RelL2_V%  : 101.22 
INFO:src.utils.logger:RelL2_P%  : 99.44 
INFO:src.utils.logger:RelL2_Fx%  : 95.43 
INFO:src.utils.logger:RelL2_Fx%  : 87.74 
INFO:src.utils.logger:Final loss left: 1.566815e-02
INFO:src.utils.logger:Final loss right: 2.196104e-02
INFO:src.utils.logger:Final loss bottom: 1.044687e-04
INFO:src.utils.logger:Final loss up: 3.649559e-02
INFO:src.utils.logger:Final loss fluid_points: 2.299547e+00
INFO:src.utils.logger:Final loss i